In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install pytorch-lightning  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 585 kB 9.0 MB/s 
     |████████████████████████████████| 419 kB 61.5 MB/s 
     |████████████████████████████████| 140 kB 73.1 MB/s 
     |████████████████████████████████| 596 kB 64.0 MB/s 
     |████████████████████████████████| 1.1 MB 57.8 MB/s 
     |████████████████████████████████| 144 kB 73.4 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
     |████████████████████████████████| 271 kB 71.2 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
pip install pytorch-forecasting

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 127 kB 6.0 MB/s 
     |████████████████████████████████| 308 kB 13.1 MB/s 
     |████████████████████████████████| 81 kB 6.9 MB/s 
     |████████████████████████████████| 209 kB 15.2 MB/s 
     |████████████████████████████████| 78 kB 7.5 MB/s 
     |████████████████████████████████| 49 kB 7.3 MB/s 
     |████████████████████████████████| 112 kB 14.4 MB/s 
     |████████████████████████████████| 146 kB 21.4 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=fa5c254c8435a0a13d203097037aa1d053054a4f28d1f5ebc0deb664d671e37a
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [4]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from pytorch_forecasting import Baseline

import joblib
import pickle
import optuna

warnings.filterwarnings('ignore')
%matplotlib inline

/usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                not been set for this class (SMAPE). The property determines if `update` by
                default needs access to the full metric state. If this is not the case, significant speedups can be
                achieved and we recommend setting this to `False`.
                We provide an checking function
                `from torchmetrics.utilities import check_forward_no_full_state`
                that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                default for now) or if `full_state_update=False` can be used safely.
                
  warnings.warn(*args, **kwargs)


### 함수정의

In [6]:
def sliding_windows(data, lookback_length, forecast_length):

    x = []
    y = []
    
    for i in range(lookback_length, len(data) - forecast_length + 1):
        _x = data[(i-lookback_length) : i]
        _y = data[i : (i + forecast_length)]
        x.append(_x)
        y.append(_y)
    return np.array(x), np.array(y)


def get_data_loader(X, y, batch_size):

    x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

    train_ds = TensorDataset(torch.Tensor(x_train), torch.Tensor(y_train))
    train_dl = DataLoader(train_ds, batch_size = batch_size)

    val_ds = TensorDataset(torch.Tensor(x_val), torch.Tensor(y_val))
    val_dl = DataLoader(val_ds, batch_size = batch_size)

    input_size = x_train.shape[-1]

    return train_dl, val_dl, input_size

### Data loading && Preproces

In [7]:
scaler = MinMaxScaler()

data=pd.read_excel('/content/drive/MyDrive/교육/AI실무인증과정/학회 발표/data/data_full.xlsx')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1858 entries, 0 to 1857
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             1858 non-null   datetime64[ns]
 1   Account DOW      1858 non-null   object        
 2   REV OBD          1858 non-null   float64       
 3   OBD NET+FSC_KRW  1858 non-null   float64       
 4   OBD A/R_KRW      1858 non-null   float64       
 5   REV CPN          1858 non-null   float64       
 6   CPN NET+FSC_KRW  1858 non-null   float64       
 7   CPN A/R_KRW      1858 non-null   float64       
 8   REV TKT          1858 non-null   float64       
 9   TKT NET+FSC_KRW  1858 non-null   float64       
 10  TKT A/R_KRW      1858 non-null   float64       
 11  WTI              1858 non-null   float64       
 12  exchanges        1858 non-null   float64       
 13  kospi            1858 non-null   float64       
 14  rates            1858 non-null   float64

In [8]:
res_data = data[data["Date"].isin(pd.date_range('2016-01-04', '2020-03-31'))]
res_data.reset_index(drop=True, inplace=True)
res_data = res_data[['Date', 'REV OBD']]
scale_cols = ['REV OBD']

In [9]:
res_data.shape

(1549, 2)

In [10]:
# 원본 코드 LSTM-pytorch_optuna_05와 다른 부분

# Loockback_period & forecasting_period
max_prediction_length = 20
lookback_length = 100
training_data_max = len(res_data) - max_prediction_length

# 학습용 데이터
data_p = res_data.iloc[:training_data_max, :]
training_data = scaler.fit_transform(data_p[scale_cols])

In [11]:
training_data_max

1529

In [12]:
data_p.shape

(1529, 2)

In [13]:
data_p

,Date,REV OBD
0,2016-01-04,28000.0
1,2016-01-05,24657.0
2,2016-01-06,26920.0
3,2016-01-07,26624.0
4,2016-01-08,28879.0
...,...,...
1524,2020-03-07,3741.0
1525,2020-03-08,4267.0
1526,2020-03-09,2385.0
1527,2020-03-10,2134.0


#### Metric 생성을 위한 oot sample 정의 (원본 코드 LSTM-pytorch_optuna_05와 다른 부분)


In [14]:
# max_prediction_length 만큼의 데이터는 예측 데이터와 비교를 위해 분리
# Training set에 없는 데이터로 구성
# Input과 output의 pair로 정의
x_for_metric = scaler.fit_transform(res_data[training_data_max -lookback_length : training_data_max][scale_cols])
y_for_metric = scaler.fit_transform(res_data[training_data_max:][scale_cols])

In [15]:
x_for_metric.shape

(100, 1)

In [16]:
y_for_metric.shape

(20, 1)

In [17]:
# LSTM은 1 step 뒤의 값만을 예측하므로, forecasting_period를 1로 두고 진행
x, y = sliding_windows(training_data, lookback_length, 1)

### Model 정의

In [18]:
# 원본 코드 LSTM-pytorch_optuna_05와 다른 부분

class LSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size,
                            num_layers = num_layers, batch_first = True)
        
        self.fc = nn.Linear(hidden_size  * num_layers, num_classes)
        
    def forward(self, x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size, device = x.device))
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size, device = x.device))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        h_out = h_out.view(-1, self.hidden_size * self.num_layers)
        out = self.fc(h_out)
        return out

In [19]:
def train(log_interval, model, train_dl, val_dl, optimizer, criterion, epoch):

    best_loss = np.inf
    for epoch in range(epoch):
        train_loss = 0.0
        model.train()
        for data, target in train_dl:

            if torch.cuda.is_available():
                data, target = data.cuda(), target.cuda()
                model = model.cuda()

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target) # mean-squared error for regression
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # validation
        valid_loss = 0.0
        model.eval()
        for data, target in val_dl:

            if torch.cuda.is_available():
                data, target = data.cuda(), target.cuda()

            output = model(data)         
            loss = criterion(output, target)
            valid_loss += loss.item()

        if ( epoch % log_interval == 0 ):
            print(f'\n Epoch {epoch} \t Training Loss: {train_loss / len(train_dl)} \t Validation Loss: {valid_loss / len(val_dl)} \n')

        if best_loss > (valid_loss / len(val_dl)):
            print(f'Validation Loss Decreased({best_loss:.6f}--->{(valid_loss / len(val_dl)):.6f}) \t Saving The Model')
            best_loss = (valid_loss / len(val_dl))
            torch.save(model.state_dict(), 'lstm_saved_model.pth')

    return best_loss


def smape(a, f):
    return 1/len(a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f))*100)

In [20]:
# 무슨 의미인가?

aa = x_for_metric
tmp = np.append( np.expand_dims(aa[1:, :], 0), np.expand_dims(y_for_metric[2, :], (0,2)), axis=1)
tmp.shape

(1, 100, 1)

In [21]:
def objective(trial):

    cfg = { 
            'batch_size' : trial.suggest_categorical('batch_size',[32, 64, 128, 256, 512]), # [64, 128, 256],
            'num_epochs' : trial.suggest_int('num_epochs', 5, 50, 5),
            'learning_rate' : trial.suggest_loguniform('learning_rate', 1e-3, 1e-1), #trial.suggest_loguniform('learning_rate', 1e-2, 1e-1), # learning rate을 0.01-0.1까지 로그 uniform 분포로 사용
            'hidden_size': trial.suggest_categorical('hidden_size',[32, 64, 128, 256, 512]),
            'num_layers': trial.suggest_int('num_layers', 1, 9, 1),       
        }

    torch.manual_seed(42)

    log_interval = 5
    num_classes = 1 # parameter에서 빼서 상수로 설정

    train_dl, val_dl, input_size = get_data_loader(x, y,  cfg['batch_size'])
    
    model = LSTM(
        num_classes = num_classes, 
        input_size = input_size, 
        hidden_size = cfg['hidden_size'], 
        num_layers = cfg['num_layers']
    )
    
    if torch.cuda.is_available():
        model = model.cuda()
        
    optimizer = optim.Adam(model.parameters(), lr=cfg['learning_rate'])
    criterion = torch.nn.MSELoss()
    best_loss = train(log_interval, model, train_dl, val_dl, optimizer, criterion, cfg['num_epochs'])

    print('best loss for the trial = ', best_loss)
    predict_data = []
    # 여기서 x는 (sample, lookback_length, 1)의 크기를 지님. 따라서, 제일 앞의 시점을 제거하려면, x[:, -1, :]이 되어야 함
    x_pred = np.expand_dims(x_for_metric, 0)  # Inference에 사용할 lookback data를 x_pred로 지정. 앞으로 x_pred를 하나씩 옮겨 가면서 inference를 할 예정

    for j, i in enumerate(range(max_prediction_length)):

        # feed the last forecast back to the model as an input
        x_pred = np.append( x_pred[:, 1:, :], np.expand_dims(y_for_metric[j, :], (0,2)), axis=1)
        xt_pred = torch.Tensor(x_pred)

        if torch.cuda.is_available():
            xt_pred = xt_pred.cuda()
        # generate the next forecast
        yt_pred = model(xt_pred)
        # tensor to array
        # x_pred = xt_pred.cpu().detach().numpy()
        y_pred = yt_pred.cpu().detach().numpy()

        # save the forecast
        predict_data.append(y_pred)

    # transform the forecasts back to the original scale
    predict_data = np.array(predict_data).reshape(-1, 1)
    SMAPE = smape(y_for_metric, predict_data)
    
    print(f' \nSMAPE : {SMAPE}')


    return SMAPE

In [22]:
sampler = optuna.samplers.TPESampler()
#   sampler = optuna.samplers.SkoptSampler()

# model.load_state_dict(torch.load('lstm_saved_model.pth'))
    
study = optuna.create_study(sampler=sampler, direction='minimize')
study.optimize(objective, n_trials= 7)
joblib.dump(study, '/content/drive/MyDrive/교육/AI실무인증과정/LSTM/LSTM_pytorch_optuna_07.pkl')

[I 2022-06-22 13:57:09,892] A new study created in memory with name: no-name-e4a8de4d-691f-4c14-b0f5-72a17dd50482



 Epoch 0 	 Training Loss: 0.1292094484799438 	 Validation Loss: 0.09185664852460225 

Validation Loss Decreased(inf--->0.091857) 	 Saving The Model
Validation Loss Decreased(0.091857--->0.079675) 	 Saving The Model
Validation Loss Decreased(0.079675--->0.073714) 	 Saving The Model

 Epoch 5 	 Training Loss: 0.008660970410952965 	 Validation Loss: 0.07807562313973904 


 Epoch 10 	 Training Loss: 0.008949390871243345 	 Validation Loss: 0.07916390616446733 


 Epoch 15 	 Training Loss: 0.00906233758562141 	 Validation Loss: 0.07971969184776147 


 Epoch 20 	 Training Loss: 0.009071836455000771 	 Validation Loss: 0.0795623358959953 


 Epoch 25 	 Training Loss: 0.00903879116392798 	 Validation Loss: 0.07922476964692275 


 Epoch 30 	 Training Loss: 0.008994861298965083 	 Validation Loss: 0.07886552096654971 



[I 2022-06-22 13:57:37,493] Trial 0 finished with value: 73.81188751634205 and parameters: {'batch_size': 128, 'num_epochs': 35, 'learning_rate': 0.0038166551199932276, 'hidden_size': 128, 'num_layers': 8}. Best is trial 0 with value: 73.81188751634205.


best loss for the trial =  0.07371366458634536
 
SMAPE : 73.81188751634205

 Epoch 0 	 Training Loss: 31.79895861148834 	 Validation Loss: 9.839898109436035 

Validation Loss Decreased(inf--->9.839898) 	 Saving The Model
Validation Loss Decreased(9.839898--->3.243505) 	 Saving The Model
Validation Loss Decreased(3.243505--->3.238394) 	 Saving The Model
Validation Loss Decreased(3.238394--->0.678338) 	 Saving The Model

 Epoch 5 	 Training Loss: 2.0326467871665956 	 Validation Loss: 1.6619821786880493 

Validation Loss Decreased(0.678338--->0.263949) 	 Saving The Model
Validation Loss Decreased(0.263949--->0.194291) 	 Saving The Model

 Epoch 10 	 Training Loss: 0.07039355337619782 	 Validation Loss: 0.13857518881559372 

Validation Loss Decreased(0.194291--->0.138575) 	 Saving The Model
Validation Loss Decreased(0.138575--->0.122744) 	 Saving The Model

 Epoch 15 	 Training Loss: 0.013757078163325786 	 Validation Loss: 0.12751669343560934 


 Epoch 20 	 Training Loss: 0.008720561023801

[I 2022-06-22 13:57:43,045] Trial 1 finished with value: 68.75950356076699 and parameters: {'batch_size': 256, 'num_epochs': 30, 'learning_rate': 0.06868989757811009, 'hidden_size': 64, 'num_layers': 8}. Best is trial 1 with value: 68.75950356076699.


...

[I 2022-06-22 13:58:39,422] Trial 5 finished with value: 65.32434635312256 and parameters: {'batch_size': 128, 'num_epochs': 30, 'learning_rate': 0.028047589553643316, 'hidden_size': 512, 'num_layers': 4}. Best is trial 5 with value: 65.32434635312256.


best loss for the trial =  0.05473890248686075
 
SMAPE : 65.32434635312256

 Epoch 0 	 Training Loss: 0.5908391961517433 	 Validation Loss: 0.07069839330183135 

Validation Loss Decreased(inf--->0.070698) 	 Saving The Model
Validation Loss Decreased(0.070698--->0.029630) 	 Saving The Model


[I 2022-06-22 13:58:43,494] Trial 6 finished with value: 74.24112833218238 and parameters: {'batch_size': 32, 'num_epochs': 5, 'learning_rate': 0.017616859610051997, 'hidden_size': 128, 'num_layers': 5}. Best is trial 5 with value: 65.32434635312256.


best loss for the trial =  0.0296303602711608
 
SMAPE : 74.24112833218238


['/content/drive/MyDrive/교육/AI실무인증과정/LSTM/LSTM_pytorch_optuna_07.pkl']

In [23]:
study = joblib.load('/content/drive/MyDrive/교육/AI실무인증과정/LSTM/LSTM_pytorch_optuna_07.pkl') 
df = study.trials_dataframe() 
df.sort_values('value')

,number,value,datetime_start,datetime_complete,duration,params_batch_size,params_hidden_size,params_learning_rate,params_num_epochs,params_num_layers,state
5,5,65.324346,2022-06-22 13:58:10.172114,2022-06-22 13:58:39.422038,0 days 00:00:29.249924,128,512,0.028048,30,4,COMPLETE
1,1,68.759504,2022-06-22 13:57:37.495193,2022-06-22 13:57:43.045597,0 days 00:00:05.550404,256,64,0.068690,30,8,COMPLETE
2,2,69.710313,2022-06-22 13:57:43.047406,2022-06-22 13:57:56.265058,0 days 00:00:13.217652,32,32,0.041194,20,5,COMPLETE
3,3,71.577363,2022-06-22 13:57:56.266987,2022-06-22 13:57:59.660447,0 days 00:00:03.393460,256,64,0.001028,40,3,COMPLETE
0,0,73.811888,2022-06-22 13:57:09.896466,2022-06-22 13:57:37.493387,0 days 00:00:27.596921,128,128,0.003817,35,8,COMPLETE
6,6,74.241128,2022-06-22 13:58:39.423770,2022-06-22 13:58:43.493808,0 days 00:00:04.070038,32,128,0.017617,5,5,COMPLETE
4,4,76.473480,2022-06-22 13:57:59.662364,2022-06-22 13:58:10.170312,0 days 00:00:10.507948,128,32,0.057983,45,6,COMPLETE


In [24]:
study.best_params

{'batch_size': 128,
 'hidden_size': 512,
 'learning_rate': 0.028047589553643316,
 'num_epochs': 30,
 'num_layers': 4}

In [25]:
optuna.visualization.plot_param_importances(study)

In [26]:
optuna.visualization.plot_optimization_history(study)

In [27]:
torch.manual_seed(42)

log_interval = 5
num_classes = 1 # parameter에서 빼서 상수로 설정

train_dl, val_dl, input_size = get_data_loader(x, y,  study.best_params['batch_size'])

model = LSTM(
    num_classes = num_classes, 
    input_size = input_size, 
    hidden_size = study.best_params['hidden_size'], 
    num_layers = study.best_params['num_layers']
)

if torch.cuda.is_available():
    model = model.cuda()
    
optimizer = optim.Adam(model.parameters(), lr=study.best_params['learning_rate'])
criterion = torch.nn.MSELoss()
best_loss = train(log_interval, model, train_dl, val_dl, optimizer, criterion, study.best_params['num_epochs'])

print('best loss for the trial = ', best_loss)
predict_data = []
# 여기서 x는 (sample, lookback_length, 1)의 크기를 지님. 따라서, 제일 앞의 시점을 제거하려면, x[:, -1, :]이 되어야 함
x_pred = np.expand_dims(x_for_metric, 0)  # Inference에 사용할 lookback data를 x_pred로 지정. 앞으로 x_pred를 하나씩 옮겨 가면서 inference를 할 예정

for j, i in enumerate(range(max_prediction_length)):

    # feed the last forecast back to the model as an input
    x_pred = np.append( x_pred[:, 1:, :], np.expand_dims(y_for_metric[j, :], (0,2)), axis=1)
    xt_pred = torch.Tensor(x_pred)

    if torch.cuda.is_available():
        xt_pred = xt_pred.cuda()
    # generate the next forecast
    yt_pred = model(xt_pred)
    # tensor to array
    # x_pred = xt_pred.cpu().detach().numpy()
    y_pred = yt_pred.cpu().detach().numpy()

    # save the forecast
    predict_data.append(y_pred)

# transform the forecasts back to the original scale
predict_data = np.array(predict_data).reshape(-1, 1)
SMAPE = smape(y_for_metric, predict_data)

print(f' \nSMAPE : {SMAPE}')


 Epoch 0 	 Training Loss: 72.56692863172955 	 Validation Loss: 11.621028582255045 

Validation Loss Decreased(inf--->11.621029) 	 Saving The Model
Validation Loss Decreased(11.621029--->6.567094) 	 Saving The Model
Validation Loss Decreased(6.567094--->4.956995) 	 Saving The Model
Validation Loss Decreased(4.956995--->1.927545) 	 Saving The Model
Validation Loss Decreased(1.927545--->0.578552) 	 Saving The Model

 Epoch 5 	 Training Loss: 0.6092309570974774 	 Validation Loss: 0.12374742080767949 

Validation Loss Decreased(0.578552--->0.123747) 	 Saving The Model
Validation Loss Decreased(0.123747--->0.057877) 	 Saving The Model

 Epoch 10 	 Training Loss: 0.016514002966384094 	 Validation Loss: 0.05473890248686075 

Validation Loss Decreased(0.057877--->0.054739) 	 Saving The Model

 Epoch 15 	 Training Loss: 0.008454732959055238 	 Validation Loss: 0.06414754316210747 


 Epoch 20 	 Training Loss: 0.008307710496915711 	 Validation Loss: 0.06501518453781803 


 Epoch 25 	 Training Los

In [28]:
# actual_data는 max_prediction_length에 해당하는 20일의 기간 : 실제 예측하려는 기간 (2020년 2월1일 ~ 2019년 2월 20일)
actual_data = data[data["Date"].isin(pd.date_range('2020-04-01', '2020-04-20'))]
actual_data['time_index'] = np.arange(len(actual_data))
actual_data['market'] = 'OBD'
actual_data.head()

,Date,Account DOW,REV OBD,OBD NET+FSC_KRW,OBD A/R_KRW,REV CPN,CPN NET+FSC_KRW,CPN A/R_KRW,REV TKT,TKT NET+FSC_KRW,TKT A/R_KRW,WTI,exchanges,kospi,rates,stock_a,stock_k,stock_kkj,time_index,market
1552,2020-04-01,WED,839.0,8.202674e+08,9.776727e+05,2907.0,1.444352e+09,496852.9593,1885.0,1.632360e+09,8.659736e+05,20.31,1231.0,1685.46,1.09,16203.0,12223.0,36241.0,0,OBD
1553,2020-04-02,THU,962.0,9.545293e+08,9.922342e+05,2169.0,1.286542e+09,593149.7392,1457.0,1.404778e+09,9.641579e+05,25.32,1228.0,1724.86,1.05,16323.0,12320.0,36045.0,1,OBD
1554,2020-04-03,FRI,1208.0,1.096642e+09,9.078166e+05,2940.0,2.063068e+09,701723.9707,1987.0,2.201680e+09,1.108042e+06,28.34,1236.0,1725.44,1.06,16299.0,12093.0,36583.0,2,OBD
1555,2020-04-04,SAT,794.0,9.508773e+08,1.197579e+06,1227.0,6.350215e+08,517539.9738,868.0,6.996180e+08,8.060116e+05,28.34,1236.0,1725.44,1.06,16299.0,12093.0,36583.0,3,OBD
1556,2020-04-05,SUN,779.0,6.990697e+08,8.973937e+05,1103.0,5.179504e+08,469583.3138,728.0,5.813863e+08,7.986076e+05,28.34,1236.0,1725.44,1.06,16299.0,12093.0,36583.0,4,OBD


In [29]:
# 실제 예측하려는 target prediction 구간 : 20일
actuals = actual_data['REV OBD']
actuals

1552     839.0
1553     962.0
1554    1208.0
1555     794.0
1556     779.0
1557     747.0
1558     501.0
1559     610.0
1560     453.0
1561    1003.0
1562     662.0
1563     585.0
1564     578.0
1565     641.0
1566     353.0
1567     328.0
1568     389.0
1569     380.0
1570     210.0
1571     331.0
Name: REV OBD, dtype: float64

In [30]:
actuals = torch.tensor(actual_data['REV OBD'].values)
print(actuals)
print(actuals.shape)

tensor([ 839.,  962., 1208.,  794.,  779.,  747.,  501.,  610.,  453., 1003.,
         662.,  585.,  578.,  641.,  353.,  328.,  389.,  380.,  210.,  331.],
       dtype=torch.float64)
torch.Size([20])


In [31]:
baseline_predictions =  torch.tensor([data.loc[data.Date == '2020-03-31']['REV OBD'].values[0]] * len(actuals)).to(torch.float32)

mae_with_baseline = (actuals - baseline_predictions).abs().mean().item()
smape_with_baseline = (2 * (baseline_predictions - actuals).abs() / (baseline_predictions.abs() + actuals.abs() + 1e-8)).mean()

print(f'mae_with_baseline : {mae_with_baseline}')
print(f'smape_with_baseline : {smape_with_baseline}')

mae_with_baseline : 858.35
smape_with_baseline : 0.8605679466244227


In [32]:
torch.tensor([data.loc[data.Date == '2020-03-31']['REV OBD'].values[0]] * len(actuals)).to(torch.float32)

tensor([1476., 1476., 1476., 1476., 1476., 1476., 1476., 1476., 1476., 1476.,
        1476., 1476., 1476., 1476., 1476., 1476., 1476., 1476., 1476., 1476.])

In [33]:
predicts = scaler.inverse_transform(predict_data)
print(predicts.shape)
print(predicts)

(20, 1)
[[1413.3187]
 [1422.5536]
 [1433.1177]
 [1437.1957]
 [1442.0737]
 [1428.227 ]
 [1427.3032]
 [1421.9257]
 [1420.4534]
 [1431.1918]
 [1419.6161]
 [1429.3292]
 [1411.4098]
 [1420.075 ]
 [1412.1228]
 [1420.3251]
 [1418.2216]
 [1417.9208]
 [1413.8599]
 [1412.155 ]]


In [34]:
predicts = torch.tensor(predicts)
predicts.shape

torch.Size([20, 1])

In [35]:
actuals = actuals.reshape(-1,1)
actuals.shape

torch.Size([20, 1])

In [36]:
mae_with_test = (actuals - predicts).abs().mean().item()
smape_with_test = smape(actuals.numpy(), predicts.numpy())

print(f'mae_with_test : {mae_with_test}')
print(f'smape_with_test : {smape_with_test}')

mae_with_test : 804.9697814941406
smape_with_test : 83.18844138697604


In [37]:
mae_with_test = (actuals - predicts).abs().mean().item()
mae_with_test 

804.9697814941406

In [ ]:
y_train.shape

np.expand_dims(pred, axis=1).shape

predicted_for_plot = sc.inverse_transform(np.concatenate((y_train, np.expand_dims(pred, axis=1))))
actual_for_plot = sc.inverse_transform(np.concatenate((y_train, y_test)))

predicted_for_plot.shape

actual_for_plot.shape

plt.plot(predicted_for_plot[300:])
plt.plot(actual_for_plot[300:])
plt.suptitle('Timeseries Prediction')
plt.axvline(x=y_train.shape[0]-300, c='r', linestyle='--')
plt.show()

NameError: ignored